In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122607")

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122607
Azure region: southcentralus
Subscription id: 19019985-4709-4996-8830-8568281fc2ff
Resource group: aml-quickstarts-122607


In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
comp_cluster = 'optimize-ml-pipeline-cluster'
comp_confg = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4, min_nodes=1)
comp_trget = ComputeTarget.create(ws, comp_cluster, comp_confg)

ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '375', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-correlation-request-id': 'cc84c4c0-064d-4c5c-8a17-15f6c7e9a6e5', 'x-ms-ratelimit-remaining-subscription-writes': '1196', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-response-type': 'standard', 'x-ms-request-id': '|00-86a529add1f0584189533373f739b90b-eadefd675621b547-00.c063121e_', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.033', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-routing-request-id': 'SOUTHCENTRALUS:20201024T115714Z:cc84c4c0-064d-4c5c-8a17-15f6c7e9a6e5', 'Date': 'Sat, 24 Oct 2020 11:57:13 GMT'}
Content: b'{"error":{"code":"BadArgument","message":"Compute name is invalid. It can include letters, digits and dashes. It must 

In [9]:
comp_trget = ws.compute_targets
for comp_name,comp_type in comp_trget.items():
    print(comp_name,comp.type,comp.provisioning_state)

project1-compute ComputeInstance Succeeded


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{
    '--batch-size' : choice(30,65,100),
    '--learning-rate' : loguniform(-7,-1)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory = './',
    compute_target = comp_trget,
    entry_script = 'train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
    max_concurrent_runs=5,
    policy=policy,
    estimator=est
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
hyperdrive_submission = exp.submit(config=hyperdrive_config)
hyperdrive_submission.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_submission.get_best_run_by_primary_metric()
print('ID : ',best_run.id)
best_metric = best_run.get_metrics()
print('Accuracy : ',best_metric['accuracy'])

final_model = best_run.register_model(model_name = 'hyper-logistic-model',model_path = '/outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###